### RSI _Opposite_Order ( RSI_O2 ) Strategy

In [1]:
import time
from datetime import datetime

import pandas as pd
import pandas_ta as ta
import numpy as np

### import csv

In [11]:
data = pd.read_csv('EURUSD_15m.csv')
data.tail()

,timestamp,open,high,low,close,adj close,volumn
7212,2022-03-01 15:45:00+00:00,1.114330,1.114579,1.113586,1.114206,1.114206,0
7213,2022-03-01 16:00:00+00:00,1.114455,1.114579,1.113462,1.113462,1.113462,0
7214,2022-03-01 16:15:00+00:00,1.113090,1.113090,1.111358,1.111605,1.111605,0
7215,2022-03-01 16:30:00+00:00,1.111729,1.111976,1.110741,1.110741,1.110741,0
7216,2022-03-01 16:45:00+00:00,1.111111,1.111111,1.110494,1.110864,1.110864,0


### create strategy

In [12]:
rsi_o2 = ta.Strategy(
    name = 'RSI 14 Days for opposite trade',
    description = 'RSI for alert',
    ta = [
        {'kind': 'ema', 'length': 200},
        {'kind': 'rsi', 'length': 24},
        {'kind': 'atr', 'length': 14}
    ]
)

In [13]:
df = data.copy()
df.ta.strategy(rsi_o2)
df.tail()

,timestamp,open,high,low,close,adj close,volumn,EMA_200,RSI_24,ATRr_14
7212,2022-03-01 15:45:00+00:00,1.114330,1.114579,1.113586,1.114206,1.114206,0,1.120351,37.721223,0.001175
7213,2022-03-01 16:00:00+00:00,1.114455,1.114579,1.113462,1.113462,1.113462,0,1.120283,35.956961,0.001171
7214,2022-03-01 16:15:00+00:00,1.113090,1.113090,1.111358,1.111605,1.111605,0,1.120197,32.055469,0.001237
7215,2022-03-01 16:30:00+00:00,1.111729,1.111976,1.110741,1.110741,1.110741,0,1.120102,30.450301,0.001237
7216,2022-03-01 16:45:00+00:00,1.111111,1.111111,1.110494,1.110864,1.110864,0,1.120011,30.965683,0.001193


### create signal

In [14]:
def backtest_rsi_o2(df, ema_len=200, rsi_len=14, atr_len=14, overbought=70, oversold=30, rr=1.5, pip_value=0.0001):
    
    rsi_o2 = ta.Strategy(
        name = 'RSI 14 Days for opposite trade',
        description = 'RSI for alert',
        ta = [
            {'kind': 'ema', 'length': ema_len},
            {'kind': 'rsi', 'length': rsi_len},
            {'kind': 'atr', 'length': atr_len}
        ]
    )
    
    df.ta.strategy(rsi_o2)
    df['action'] = np.nan

    orders = pd.DataFrame(columns = ['action', 'open time', 'open', 'close time', 'close', 'T/P', 'S/L','result'])
    current_action = 'close'
    order = {}
    unknow_result = []

    for index, row in df.iterrows():
        if current_action == 'close':
            
            if row['EMA_'+str(ema_len)] < row['low'] and row['RSI_'+str(rsi_len)] > overbought:
                row['action'] = 'buy'
                order['action'] = 'buy'
                order['open time'] = row['timestamp']
                order['open'] = row['open']
                order['S/L'] = row['low'] - row['ATRr_'+str(atr_len)]
                order['T/P'] = row['open'] + (rr * abs(row['open'] - order['S/L']))
                current_action = 'buy'
            
            elif row['EMA_'+str(ema_len)] > row['high'] and row['RSI_'+str(rsi_len)] < oversold:   
                row['action'] = 'sell'
                order['action'] = 'sell'
                order['open time'] = row['timestamp']
                order['open'] = row['open']
                order['S/L'] = row['high'] + row['ATRr_'+str(atr_len)]
                order['T/P'] = row['open'] - (rr * abs(row['open'] - order['S/L']))
                current_action = 'sell'
 
        if current_action == 'buy':
            if row['low'] <= order['S/L'] and row['high'] >= order['T/P']:
                unknow_result.append(row)
            if row['low'] <= order['S/L']:
                order['result'] = 'S/L'
                order['close time'] = row['timestamp']
                order['close'] = order['S/L']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}
            elif row['high'] >= order['T/P']:
                order['result'] = 'T/P'
                order['close time'] = row['timestamp']
                order['close'] = order['T/P']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}
                
        elif current_action == 'sell':
            if row['high'] >= order['S/L'] and row['low'] <= order['T/P']:
                unknow_result.append(row)
            if row['high'] >= order['S/L']:
                order['result'] = 'S/L'
                order['close time'] = row['timestamp']
                order['close'] = order['S/L']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}
            elif row['low'] <= order['T/P']:
                order['result'] = 'T/P'
                order['close time'] = row['timestamp']
                order['close'] = order['T/P']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}

    try:
        win_rate = round(len(orders[orders['result'] == 'T/P']) * 100 / len(orders), 2)
        loss_rate = round(len(orders[orders['result'] == 'S/L']) * 100 / len(orders), 2)

        win_orders = int((win_rate * len(orders)) / 100 )
        loss_orders = len(orders) - win_orders
        gain = (win_orders * rr) - loss_orders
        
        result_info = f"total orders: {len(orders)}, overbought: {overbought}, oversold: {oversold} ,rr: {rr}\nema: {ema_len}, rsi: {rsi_len}, atr: {atr_len}, gain: {gain}\n"
        return {'win_rate': win_rate, 'info': result_info, 'total_order': len(orders), 'rr': rr, 'gain': gain, 'unknow': unknow_result}
    except:
#         print("No orders to action")
        return {'win_rate': 0, 'info': 'No orders to action'}
   

In [15]:
orders = backtest_rsi_o2(df, ema_len=200, rsi_len=14, atr_len=14, overbought=70, oversold=30, rr=1.5)
# print(orders['win_rate'], '%,', orders['info'])
# 69.7 %, total orders: 165, overbought: 69, oversold: 31 ,rr: 1.5
# ema: 200, rsi: 14, atr: 14
print(f"{orders['win_rate']} %, {orders['info']}")

67.22 %, total orders: 180, overbought: 70, oversold: 30 ,rr: 1.5
ema: 200, rsi: 14, atr: 14, gain: 120.0



In [16]:
overB = 95
overS = 5
ema_list = [200] # list(range(100,210, 10))
rsi_list = list(range(10, 20+1))
atr_list = list(range(10, 20+1))
params_list = []

for ema in ema_list:
    for rsi in rsi_list:
        for atr in atr_list:
            for i in range(40):
                params_list.append([ema, rsi, atr, overB-i, overS+i, 1.5])
# params_list[5:10]
len(params_list)

4840

In [17]:
good_params = {'win': 0, 'info': ''}
start = 0
end = 4840
win_start = 0

loop_num = start
for param in params_list[start:end]:
    result = backtest_rsi_o2(df, ema_len=param[0], rsi_len=param[1], atr_len=param[2], overbought=param[3], oversold=param[4], rr=param[5])
    if result['win_rate'] > win_start and result['total_order'] > 30:
        win_start = result['win_rate']
        good_params['win'] = result['win_rate']
        good_params['info'] = result['info']
    loop_num = loop_num + 1
    print(f"run: {loop_num} to {end}", end = "\r")
    
print(f"\nwin rate: {good_params['win']} %\n{good_params['info']}")

run: 4840 to 4840
win rate: 85.71 %
total orders: 35, overbought: 74, oversold: 26 ,rr: 1.5
ema: 200, rsi: 20, atr: 13, gain: 37.5



In [12]:
# EURUSD 1 pip = 0.0001
# GBPUSD 1 pip = 0.0001
# USDJPY 1 pip = 0.01
# AUDUSD 1 pip = 0.0001
# 7.59 * 0.0001 == 0.000759
0.000759 / 0.01

0.0759

run: 1 to 2000
* win rate: 67.38 %
* total orders: 233, overbought: 71, oversold: 29 ,rr: 1.5
* ema: 200, rsi: 11, atr: 11, gain: 157.0

run: 2000 to 4000
* win rate: 66.67 %
* total orders: 159, overbought: 71, oversold: 29 ,rr: 1.5
* ema: 200, rsi: 15, atr: 19, gain: 106.0

run: 4000 to 4840
* win rate: 64.29 %
* total orders: 56, overbought: 73, oversold: 27 ,rr: 1.5
* ema: 200, rsi: 20, atr: 10, gain: 34.0

In [20]:
win_rate = 75.61
total_orders = 41
rr = 1.5
win_orders = int((win_rate * total_orders) / 100 )
loss_orders = total_orders - win_orders
gain = (win_orders * rr) - loss_orders
gain

36.5

#### The best params for EURUSD 15m

* win rate: 85.71 %
* total orders: 35, overbought: 74, oversold: 26 ,rr: 1.5
* ema: 200, rsi: 20, atr: 13
* gain: 37.5

#### The best params for GBPUSD 15m


#### The best params for USDJPY 15m



#### The best params for AUDUSD 15m

* win rate: 67.38 %
* total orders: 233, overbought: 71, oversold: 29 ,rr: 1.5
* ema: 200, rsi: 11, atr: 11 
* gain: 157.0

### backtest

#### === Todo ===

- &check; import forex data from yohoo finance GBPUSD, EURUSD, USDJPY TF: 5min, 15min, 30min, 1hr, 4hr.
- &check; backtest RSI_O2 strategy with forex.
- &check; get data from binance BTCUSDT, ETHUSDT, BNBUSDT, ADAUSDT, SHIBUSDT, GALAUSDT, UNIUSDT, CAKEUSDT, SOLUSDT, LUNAUSDT, DOTUSDT
   TF: 15min, 30min, 1hr, 4hr, 1day
- &check; create EMA_3_lines strategy.
- &check; backtest EMA_3_lines strategy with forex.
- &check; create bot pull data from yahoo finance because they can give us last 2 months.
